# Customizing the Django Admin

* [Reference](https://testdriven.io/blog/customize-django-admin/)

Django's automatically generated `admin` site is one of the biggest strengths of the framework. 

In this Chapter, we will discuss about

- Perform basic Django admin site configuration
- Explain how Django model attributes affect the admin site
- Use `list_display` to control which model fields are displayed
- Add custom fields to `list_display` and format existing ones
- Add links to related model objects in `list_display`
- Enable search and filters via `search_fields` and `list_filter`
- Handle model inlines for both `N:1` and `M:M` relationships
- Use Django admin actions and create custom ones
- Override Django admin forms and templates
- Utilize DjangoQL for advanced searching functionality
- Import data in and export data to different formats using `django-import-export`
- Modify the appearance of your admin site via `django-admin-interface`

## Basic Admin Site Customization

- The `Django admin site` provides some basic configuration options. These options allow you to **change the site's title, header, site URL, and more**. 

- The `admin.site` settings are usually modified in your **project's main `urls.py`** file.

    ```python
    # django_project/urls.py

    ...
    admin.site.site_title = "Blog Site Admin (DEV)"
    admin.site.site_header = "Blog Administration"
    admin.site.index_title = "Site Administration"
    ...
    ```
- Another thing you should do is **change the `default /admin` URL**. This'll make it more difficult for malicious actors to find your admin panel.

    ```python
    # django_project/urls.py
    urlpatterns = [
        path("secretadmin/", admin.site.urls),
        path('users/', include('users.urls')),
        path('', include('blog.urls')),
    ] 

    ...
    ```
- Your admin site should now be accessible at `http://localhost:8000/secretadmin`.

## Django Model and Admin

Some `Django model attributes` directly affect the `Django admin site`. Most importantly:

- `__str__()` is used to define object's display name
  
- `Meta` class is used to set various metadata options (e.g., `ordering` and `verbose_name`)

Here's an example of how these attributes are used in practice:

- By providing the `ordering` attribute the categories are now ordered by `date_posted`.

    ```python
    class Post(models.Model):
        title = models.CharField(max_length=80)
        author = models.ForeignKey(User, on_delete= models.CASCADE)
        content = models.TextField()
        date_posted = models.DateTimeField(default=timezone.now)

        def __str__(self):
            return self.title
        
        class Meta:
            verbose_name = "Blog Post"
            verbose_name_plural = "Blog Posts"
            ordering = ["-date_posted"]
        
        def get_absolute_url(self):
            return reverse('post-detail', kwargs={'pk': self.pk})
    ```

Here I list more commonly used `Meta`class options, check it out.

<table>
    <thead>
        <tr>
            <th>Meta Option</th>
            <th>Purpose</th>
            <th>Code Example</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>verbose_name</td>
            <td>Sets a human-readable singular name for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;verbose_name = "Blog Post"<br></code></td>
        </tr>
        <tr>
            <td>verbose_name_plural</td>
            <td>Sets a human-readable plural name for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;verbose_name_plural = "Blog Posts"<br></code></td>
        </tr>
        <tr>
            <td>ordering</td>
            <td>Specifies the default ordering of model instances.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;ordering = ['-date_posted']<br></code></td>
        </tr>
        <tr>
            <td>unique_together</td>
            <td>Enforces a unique constraint on the specified fields.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;unique_together = ('author', 'title')<br></code></td>
        </tr>
        <tr>
            <td>permissions</td>
            <td>Defines custom permissions for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;permissions = [('can_publish', 'Can publish posts')]<br></code></td>
        </tr>
        <tr>
            <td>db_table</td>
            <td>Sets the name of the database table used for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;db_table = 'custom_table_name'<br></code></td>
        </tr>
        <tr>
            <td>get_latest_by</td>
            <td>Defines the default field used when retrieving the latest object.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;get_latest_by = 'date_posted'<br></code></td>
        </tr>
        <tr>
            <td>indexes</td>
            <td>Creates database indexes on specified fields for faster lookups.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;indexes = [models.Index(fields=['date_posted'])]<br></code></td>
        </tr>
        <tr>
            <td>abstract</td>
            <td>Specifies that the model is abstract and will not be created as a database table.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;abstract = True<br></code></td>
        </tr>
        <tr>
            <td>constraints</td>
            <td>Defines custom database constraints on the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;constraints = [models.UniqueConstraint(fields=['title'], name='unique_title')]<br></code></td>
        </tr>
        <tr>
            <td>default_permissions</td>
            <td>Specifies the default permissions (add, change, delete, view) for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;default_permissions = ['add', 'change', 'view']<br></code></td>
        </tr>
        <tr>
            <td>proxy</td>
            <td>Creates a proxy model that inherits from another model, used for different model behavior.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;proxy = True<br></code></td>
        </tr>
        <tr>
            <td>managed</td>
            <td>Indicates if Django should manage the model’s database table (e.g., create/drop tables).</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;managed = False<br></code></td>
        </tr>
        <tr>
            <td>auto_created</td>
            <td>Indicates that the model was automatically created (typically for intermediate tables).</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;auto_created = True<br></code></td>
        </tr>
        <tr>
            <td>select_on_save</td>
            <td>Forces Django to select the instance from the database immediately after saving.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;select_on_save = True<br></code></td>
        </tr>
        <tr>
            <td>default_related_name</td>
            <td>Sets the default reverse relationship name for foreign key fields.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;default_related_name = 'related_posts'<br></code></td>
        </tr>
        <tr>
            <td>index_together</td>
            <td>Creates a composite index on the specified fields for efficient multi-column searches.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;index_together = [['field1', 'field2']]<br></code></td>
        </tr>
        <tr>
            <td>apps</td>
            <td>Customizes the application labels associated with the model (used in advanced scenarios).</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;apps = ['app1', 'app2']<br></code></td>
        </tr>
        <tr>
            <td>base_manager_name</td>
            <td>Specifies the name of the default manager for the model.</td>
            <td><code>class Meta:<br> &nbsp;&nbsp;base_manager_name = 'custom_manager'<br></code></td>
        </tr>
    </tbody>
</table>


## Customize Admin Site with ModelAdmin Class

In this section, we'll take a look at how to use the `ModelAdmin` class to customize the admin site.

### Control List Display

- The `list_display` attribute allows you to **control which model fields are displayed on the model list page**. 

- Another great thing about it is that it can display related model fields using the `__` operator.

- Here's the demo script 

    ```python
    # blog/admin.py

    @admin.register(Post)
    class PostAdmin(admin.ModelAdmin):
        # Fields to display in the list view
        list_display = ('title', 'author', 'date_posted')  # Displays these fields in the list view
        # Default ordering of records
        ordering = ('-date_posted',)  # Orders the posts by date_posted in descending order
        # Read-only fields in the form
        readonly_fields = ('date_posted',)  # Makes date_posted field read-only

    # Custom admin for the Comment model
    @admin.register(Comment)
    class CommentAdmin(admin.ModelAdmin):
        # Fields to display in the list view
        list_display = ('post', 'author', 'date_posted')  # Displays post, author, and date_posted in list view
        # Default ordering of records
        ordering = ('-date_posted',)  # Orders comments by date_posted in descending order
        # Read-only fields in the form
        readonly_fields = ('date_posted',)  # Makes date_posted field read-only
    ```

### List Display Custom Fields

- The `list_display` setting can also be used to add custom fields. 
  
- To add a custom field, you must **define a new method** within the `ModelAdmin` class.
  
- Here's the demo script

    ```python
    # blog/admin.py
    @admin.register(Post)
    class PostAdmin(admin.ModelAdmin):
        # Fields to display in the list view
        list_display = ('title', 'author', 'date_posted','comment_count')  # Displays these fields in the list view
        # ...
        
        # Method to count comments for each post
        def comment_count(self, obj):
            return obj.comments_count
        comment_count.short_description = 'Number of Comments'  # Sets the column header name in the admin
        
        # Optimizing query with annotation
        def get_queryset(self, request):
            queryset = super().get_queryset(request)
            # Annotate each post with the count of related comments
            queryset = queryset.annotate(comments_count=Count('comment'))
            return queryset
    ```

- Explanation
- `comment_count(self, obj)` Method:
  - This method is used to display the **number of comments related to each post**.
  - It uses the `comments_count` value annotated to each post in the `get_queryset` method.
  - `short_description` is used to **define the label** that will appear as the column header in the admin list view.
- `get_queryset(self, request)`:
  - This method overrides the default `get_queryset` of the `ModelAdmin` to annotate each post with the count of its related comments.
  - The `annotate(comments_count=Count('comment'))` line adds a `comments_count` field to each post object, counting the related Comment instances.

### Link Related Model Objects

- Sometimes it can be helpful to **add links to related model objects** instead of just showing their display name

- Before we do that, let's take a look at the Django admin site URL structure:

    <table>
        <thead>
            <tr>
                <th>Page</th>
                <th>URL</th>
                <th>Description</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>List</td>
                <td><code>admin:&lt;app&gt;_&lt;model&gt;_changelist</code></td>
                <td>Displays the list of objects</td>
            </tr>
            <tr>
                <td>Add</td>
                <td><code>admin:&lt;app&gt;_&lt;model&gt;_add</code></td>
                <td>Object add form</td>
            </tr>
            <tr>
                <td>Change</td>
                <td><code>admin:&lt;app&gt;_&lt;model&gt;_change</code></td>
                <td>Object change form (requires <code>objectId</code>)</td>
            </tr>
            <tr>
                <td>Delete</td>
                <td><code>admin:&lt;app&gt;_&lt;model&gt;_delete</code></td>
                <td>Object delete form (requires <code>objectId</code>)</td>
            </tr>
            <tr>
                <td>History</td>
                <td><code>admin:&lt;app&gt;_&lt;model&gt;_history</code></td>
                <td>Displays object's history (requires <code>objectId</code>)</td>
            </tr>
        </tbody>
    </table>
<br>

- To demonstrate how this is done, we'll link author change url on the comment list page, here's the demo script
- We used the `reverse` method to reverse the URL and passed `obj.author.id` as the `objectId`.

    ```python
    @admin.register(Comment)
    class CommentAdmin(admin.ModelAdmin):
        # Fields to display in the list view
        list_display = ('post', 'date_posted', 'display_author')  # Adds the custom display_author method to list_display
        #...
        
        # Use select_related to optimize queries and reduce the number of database hits
        list_select_related = ["author"]

        # Custom method to display the author field as a clickable link
        def display_author(self, obj):
            # Generates the URL for the change form of the related author using its primary key
            link = reverse("admin:auth_user_change", args=[obj.author.id])
            # Returns an HTML anchor tag (<a>) with the link, making the author's name clickable
            return format_html('<a href="{}">{}</a>', link, obj.author)
        
        # Sets the column header name for the custom method in the list view
        display_author.short_description = "Author"
    ```

### Filter Model Objects

Django admin makes it easy to filter objects. 

- Best of all, **Django can stack filters** -- e.g., filter by two or more fields simultaneously.
  
- To filter by a related object's fields, use the `__` operator.
  
- For more advanced filtering functionality, you can also define `custom filters`. 
  - To define a custom filter, you must specify the options or so-called `lookups` and a `queryset` for each `lookup`

- Here's the demo script to filter `Comment` by `posted_date` of `Post`
  
  ```python
  from django.utils import timezone
  from datetime import datetime, timedelta

  # Custom filter to display post's posted date with a custom name
  class PostPostedDateFilter(admin.SimpleListFilter):
      title = 'Post Posted Date'  # Sets the display title for the filter
      parameter_name = 'post__date_posted'  # The field name used in the query parameters

      def lookups(self, request, model_admin):
          """
          Returns a list of tuples. Each tuple contains a value and a display name for the filter options.
          """
          return [
              ('today', 'Today'),
              ('past_7_days', 'Past 7 days'),
              ('this_month', 'This month'),
              ('this_year', 'This year'),
          ]

      def queryset(self, request, queryset):
          """
          Filters the queryset based on the selected filter option.
          """
          today = timezone.now().date()
          
          if self.value() == 'today':
              return queryset.filter(post__date_posted__date=today)
          elif self.value() == 'past_7_days':
              past_7_days = today - timedelta(days=7)
              return queryset.filter(post__date_posted__date__gte=past_7_days)
          elif self.value() == 'this_month':
              return queryset.filter(
                  post__date_posted__year=today.year,
                  post__date_posted__month=today.month
              )
          elif self.value() == 'this_year':
              return queryset.filter(post__date_posted__year=today.year)
          return queryset  # Default returns all if "Any date" is selected or no option matches

  # Custom admin for the Comment model
  @admin.register(Comment)
  class CommentAdmin(admin.ModelAdmin):
      # ...
      # Fields to filter the records
      list_filter = ('author', 'date_posted',PostPostedDateFilter)  # Adds filters for post, author, and date posted
      # ...
  ```